### Data labeling
In this notebook we will be attempting to label scraped data from several e-commerce websites to later use the labels to train models. We will be associating semantical label to the data. This way, we will be able to test model's performance at classifying tags on the same sites, and also the model's ability to generalize to other websites. 


We will be assessing the extracted data's quality and giving it the following semantic labels, relevant to e-commerce websites:
* list/detail_price
* list/detail_description
* list/detail_name
* list/detail_image
* product container

The price, description and name are self-explanatory, having a detail/list version representing whether the data is on a list or on a detail page. We will also be extracting *product containers*. They represent boxes which encapsulate a products data and would help associate information into cohesive groups.

![Olx label example](imgs/labelvis.png)

In [1]:
%matplotlib inline
import sys, os
import re

# lxml
from lxml import etree

# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# add the library path
sys.path.append(os.path.join(os.getcwd(), "src"))
from features import extract_features_from_html, extract_features_from_df

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [2]:
# print some statistics
WEBSITES = ['emag', 'okazii', 'olx', 'aliexpress', 'amazon', 'lajumate', 'piata-az']
FILENAMES = ['../data/ecommerce/' + name + '.csv' for name in WEBSITES]
NO_OF_PAGES = 370  # the number of pages at which to truncate the 

dfs = {}

# truncate the number of pages to 350
# print each file and how many pages 
print('number of pages per site(truncated to {0})'.format(NO_OF_PAGES))
print('='*80)
for site, filename in zip(WEBSITES, FILENAMES):
    df = pd.read_csv(filename)
    print('{0}: {1} pages'.format(site, df.size))
    dfs[site] = df[:NO_OF_PAGES]  
    
dfs['amazon'].head()

number of pages per site(truncated to 370)
emag: 390 pages
okazii: 374 pages
olx: 482 pages
aliexpress: 372 pages
amazon: 598 pages
lajumate: 372 pages
piata-az: 3110 pages


,html,url
0,"<!doctype html><html class=""a-no-js"" data-19ax...",https://www.amazon.com/stream
1,"<!doctype html><html class=""a-no-js"" data-19ax...",https://www.amazon.com/ref=nav_logo/131-623793...
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!doctype ...,https://www.amazon.com/gp/site-directory/ref=n...
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,https://www.amazon.com/gp/customer-preferences...
4,\n\n\n\n\n\n\n\n \n \n\n\n\n\n\n\n\n\n\n\n\n...,https://www.amazon.com/gp/product/B00DBYBNEE/r...


In [3]:
# writing the data to a file
def concat_and_write(dfs):
    pd.concat(dfs.values(), axis='rows', ignore_index=True).to_csv('../data/ecommerce.csv')
    
print('concatenating and writing the dataframes')
concat_and_write(dfs)

concatenating and writing the dataframes


### Labeling
In this section, we will be labeling tags on all pages of a website based on url filters to select list and detail pages. For example:
`https://www.olx.ro/oferta/jante-mercedes-amg-ID8Vi7d.html` is a detail page. On each such page we will apply a custom-made XPath to extract the desired elements. The order of the labels is given by `lxml` iteration order so it's consistent with the feature extraction step.

In [4]:
def match_urls(df, url_regex):
    """Returns a datarame of all the urls, telling whther they are matched or not"""
    url_pattrn = re.compile(url_regex)
    url_filter = lambda url: bool(url_pattrn.match(url))
    
    df['url_match'] = df['url'].apply(url_filter)
    df['tree'] = df['html'].apply(etree.HTML)  # get the nodes
    
    return df


def label_pages(df, url_regex, rules):
    """Returns a DataFrame wherhe each row represents a tag 
    and there are columns that specify whther the tag has a label or not.
    
    Rules is a dictionary of {label: xpath} which tells the xpath to match
    tags with the corresponding label.
    """
    df = match_urls(df, url_regex)  # match the urls
    
    for label, xpath in rules.items():
        # add columns with the tags matching the xpaths
        label_tag_col_name = '{0}_label_tags'.format(label)
        df[label_tag_col_name] = None  # set it to the list of tags
        
        # the series of lists of tags that match the rule
        # applied only to matching ulrs
        tag_series = df.loc[df['url_match'], 'tree'].apply(lambda tree: tree.xpath(xpath))
        df.loc[df['url_match'], label_tag_col_name] = tag_series
    
    # iterate over rows, explode into component tags and mark them 
    # as being labeled or not
    pages_labels = []  # the row values
    for row_data in df.iterrows():
        row = row_data[1]  # ignore the index, which is on the first pos
        
        # a series of the tags
        tags_series = pd.Series(list(row['tree'].iter()))  
        label_cols = []
        
        for label, xpath in rules.items():
            label_tag_col_name = '{0}_label_tags'.format(label)
            label_col_name = '{0}_label'.format(label)
            
            if row['url_match']:
                # if the url is among the sought ones
                # chekc if the tags are in the current list of xpath-ed tags
                label_series = tags_series.apply(lambda tag: tag in row[label_tag_col_name])
                
            else:
                label_series = pd.Series(data=np.zeros(tags_series.size, dtype=bool))
                
            label_series.name = label_col_name  # rename it to the final name
            label_cols.append(label_series)
            
        # cocnatenate the labels for the current page
        page_label_df = pd.concat(label_cols, axis='columns')
        #page_label_df['url'] = row['url']  # add the url
        pages_labels.append(page_label_df)
    
    # return the veritcally stacked values of all the pages
    return pd.concat(pages_labels, axis='rows', ignore_index=True)


def label_data(df, rules):
    """Returns a df of all the labels given a set of url filters
    and tag xpaths"""
    return pd.concat([label_pages(df, rule['url_regex'], rule['xpaths']) for rule in rules], axis='columns')
    

In [5]:
# print urls matching the pattern
# just the first 10 for a quick sanity-check

df_match = match_urls(dfs['olx'], r'.*oferta.*')
print('\n'.join(df_match[df_match['url_match']].iloc[1:10].url))

https://www.olx.ro/oferta/vand-carucior-sport-baby-design-lupo-comfort-3in1-cu-scoica-leo-0m-gr-ID6SHnm.html
https://www.olx.ro/oferta/ford-focus-1-6-tdci-volan-dreapta-ID8SF3J.html
https://www.olx.ro/oferta/angajam-sef-de-restaurant-4-000-lei-in-mana-ID8klH3.html
https://www.olx.ro/oferta/montaj-aer-conditionat-ID918T7.html
https://www.olx.ro/oferta/munca-necalificata-in-slovacia-cu-contract-de-munca-comision-0-id-32-ID8Cvuf.html
https://www.olx.ro/oferta/pc-intel-dual-core-e3300-2-5ghz-4gbddr3-160gb-dvdrw-dell-380-tower-ID83AhX.html
https://www.olx.ro/oferta/regenerare-curatare-filtru-de-particule-dpf-fap-ID8etJl.html
https://www.olx.ro/oferta/vand-duplex-borhanci-ID8QZn9.html
https://www.olx.ro/oferta/vanzare-apartament-3-cam-77-mp-utili-in-manastur-ID70f1I.html


In [6]:
# same
url_regex = r'.*oferta.*'
rules =  {'detail_price': r'//*[contains(concat( " ", @class, " " ), concat( " ", "arranged", " " ))]',
          'detail_title': r'//h1'}
df = label_pages(dfs['olx'], url_regex, rules)

# print hte pages that have the either of the labels
df[df['detail_title_label'] | df['detail_price_label']].head()

,detail_price_label,detail_title_label
244751,False,True
244930,True,False
245046,True,False
245585,False,True
246464,False,True


### Labeling the tags
From here on, we will be labeling tags by filtering urls from each site with regular expressions and extracting relevant tags with xpaths.

#### OLX

In [1]:
# define the rules
rules =  [
            {
                'url_regex': r'.*oferta.*',
                'xpaths': 
                {
                    'detail_price': r'//*[contains(concat( " ", @class, " " ), concat( " ", "arranged", " " ))]',
                    'detail_title': r'//h1',
                    'detail_description': r'//*[(@id = "textContent")]//*[contains(concat( " ", @class, " " ), concat( " ", "large", " " ))]',
                    'detail_image': r'//*[contains(concat( " ", @class, " " ), concat( " ", "scale4", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "fleft", " " ))]'
                }
            },
            {
                'url_regex': r'((?!oferta).)*',
                'xpaths': 
                {
                    'list_container': r'//td[contains(@class,"offer")]',
                    'list_title': r'//td[contains(@class,"offer")]//a/strong',
                    'list_price': r'//td[contains(@class,"offer")]//p[contains(@class,"price")]/strong',
                    'list_image': r'//*[contains(concat( " ", @class, " " ), concat( " ", "scale4", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "fleft", " " ))]'
                }
            }
        ]

# extract from them
olx_df = label_data(dfs['olx'], rules)

'[{"url_regex": ".*oferta.*", "xpaths": {"detail_price": "//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"arranged\\", \\" \\" ))]", "detail_title": "//h1", "detail_description": "//*[(@id = \\"textContent\\")]//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"large\\", \\" \\" ))]", "detail_image": "//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"scale4\\", \\" \\" ))]//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"fleft\\", \\" \\" ))]"}}, {"url_regex": "((?!oferta).)*", "xpaths": {"list_container": "//td[contains(@class,\\"offer\\")]", "list_title": "//td[contains(@class,\\"offer\\")]//a/strong", "list_price": "//td[contains(@class,\\"offer\\")]//p[contains(@class,\\"price\\")]/strong", "list_image": "//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"scale4\\", \\" \\" ))]//*[contains(concat( \\" \\", @class, \\" \\" ), concat( \\" \\", \\"fleft\\", \\" \\" ))]"}}]'

In [8]:
# sum all tags to see a total
olx_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label            42
detail_title_label            83
detail_description_label      83
detail_image_label           210
list_container_label        5493
list_title_label            5231
list_price_label            3745
list_image_label            4156
dtype: int64

#### Emag

In [9]:
rules =  [
            {
                'url_regex': r'.*/pd/[A-Z0-9]*/.*',
                'xpaths': 
                {
                    'detail_price': r'//*[contains(concat( " ", @class, " " ), concat( " ", "product-page-pricing", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "product-new-price", " " ))]',
                    'detail_title': r'//*[contains(concat( " ", @class, " " ), concat( " ", "page-title", " " ))]',
                    'detail_description': r'//*[(@id = "description-body")]',
                    'detail_image': r'//*[(@id = "product-gallery")]//img'
                }
            },
            {
                'url_regex': r'.*',
                'xpaths': 
                {
                    'list_container': r'//div[contains(@class, "card-section-wrapper")]',
                    'list_title': r'//*[contains(concat( " ", @class, " " ), concat( " ", "product-title", " " ))]',
                    'list_price': r'//*[contains(concat( " ", @class, " " ), concat( " ", "product-new-price", " " ))]',
                    'list_image': r'//*[(@id = "card_grid")]//img'
                }
            }
        ]

# extract from them
emag_df = label_data(dfs['emag'], rules)

In [10]:
emag_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label           19
detail_title_label           19
detail_description_label     17
detail_image_label           19
list_container_label          0
list_title_label            513
list_price_label            530
list_image_label              0
dtype: int64

#### Lajumate

In [11]:
rules =  [
            {
                'url_regex': r'^((?!anunturi).)+$',
                'xpaths': 
                {
                    'detail_price': r'//*[(@id = "price")]',
                    'detail_title': r'//*[@id="main_info_holder"]//h1',
                    'detail_description': r'//*[(@id = "description")]//p',
                    'detail_image': r'//*[(@id = "holder_popup")]//img'
                }
            },
            {
                'url_regex': r'.*anunturi.*',
                'xpaths': 
                {
                    'list_container': r'//*[@id="list_cart_holder"]//*[contains(concat( " ", @class, " " ), concat( " ", "item_cart", " " ))]',
                    'list_title': r'//*[@id="list_cart_holder"]//*[contains(@class,"title")]',
                    'list_price': r'//*[(@id = "list_cart_holder")]//*[contains(concat( " ", @class, " " ), concat( " ", "shadow", " " ))]',
                    'list_image': r'//*[(@id = "image_holder")]//img'
                }
            }
        ]

# extract from them
lajumate_df = label_data(dfs['lajumate'], rules)

In [12]:
lajumate_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label            83
detail_title_label            82
detail_description_label      89
detail_image_label           460
list_container_label        2189
list_title_label            2189
list_price_label            2147
list_image_label               0
dtype: int64

In [13]:
print('\n'.join(dfs['lajumate'].url))

https://lajumate.ro
https://lajumate.ro/anunt/nou
https://lajumate.ro/anunturi_auto-moto-nautic.html
https://lajumate.ro/anunturi_imobiliare.html
https://lajumate.ro/anunturi_electronice-electrocasnice.html
https://lajumate.ro/anunturi_casa-gradina.html
https://lajumate.ro/anunturi_moda-frumusete.html
https://lajumate.ro/anunturi_mamici-si-pitici.html
https://lajumate.ro/anunturi_afaceri-servicii.html
https://lajumate.ro/anunturi_locuri-de-munca.html
https://lajumate.ro/anunturi_animale-agro-industrie.html
https://lajumate.ro/anunturi_sport-hobby-timp-liber.html
https://lajumate.ro/angajam-receptionera-pensiune-5078058.html
https://lajumate.ro/vand-cartofi-mov-blue-congo-soi-superior-de-cartofi-167252.html
https://lajumate.ro/strungari-5866549.html
https://lajumate.ro/coordonator-sectie-productie-5928882.html
https://lajumate.ro/redactori-online-spynewsro-5869923.html
https://lajumate.ro/ceas-tissot-5910144.html
https://lajumate.ro/casa-plus-teren-in-curtea-de-arges-an-2016-5415148.htm

#### Okazii

In [14]:
rules =  [
            {
                'url_regex': r'.*a[0-9]+$',
                'xpaths': 
                {
                    'detail_price': r'//*[@id="buy_form"]/div[1]/div[2]/span/span/span[1]/span/span[1]',
                    'detail_title': r'//*[contains(concat( " ", @class, " " ), concat( " ", "fn", " " ))]',
                    'detail_description': r'//*[(@id = "description_tmce")]',
                    'detail_image': r'//*[(@id = "gallery_show_big_img")]'
                }
            },
            {
                'url_regex': r'.*[^0-9][0-9]{0,5}$',
                'xpaths': 
                {
                    'list_container': r'//div[contains(@class, "list-item")]',
                    'list_title': r'//*[contains(@class,"item-title")]//h2//span',
                    'list_price': r'//*[contains(@class,"prSup")][span]',
                    'list_image': r'//*[contains(concat( " ", @class, " " ), concat( " ", "ajaxTrackable", " " ))]//img'
                }
            }
        ]

# extract from them
okazii_df = label_data(dfs['okazii'], rules)

In [15]:
okazii_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label             0
detail_title_label             0
detail_description_label       0
detail_image_label             0
list_container_label        7766
list_title_label            7128
list_price_label            7756
list_image_label            7720
dtype: int64

**NOTE:** Apparently the okazii data doesn't have any detail pages.

#### Piata-AZ

In [16]:
rules =  [
            {
                'url_regex': r'.*anunturi.*[0-9]+$',
                'xpaths': 
                {
                    'detail_price': r'//*[@id="detaliu-pret"]',
                    'detail_title': r'//h1',
                    'detail_description': r'//*[@id="anunt-descriere"]/p[1]',
                    'detail_image': r'//*[@id="slider"]/div/ul/li[1]/a/img'
                }
            },
            {
                'url_regex': r'.*(page|cautare).*',
                'xpaths': 
                {
                    'list_container': r'//div[contains(@class, "anunt-promo")]',
                    'list_title': r'//div[contains(@class, "anunt-promo")]//h3',
                    'list_price': r'//div[contains(@class, "anunt-promo")]//div[contains(@class, "pret")]',
                    'list_image': r'//div[contains(@class, "anunt-promo")]//div[contains(@class, "pic")]//img'
                }
            }
        ]

piataaz_df = label_data(dfs['piata-az'], rules)

In [17]:
piataaz_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label           9
detail_title_label          47
detail_description_label    12
detail_image_label           0
list_container_label        89
list_title_label            89
list_price_label            56
list_image_label            51
dtype: int64

**NOTE:** images are missing because they are loaded with ajax.

#### Aliexpress

In [18]:
rules =  [
            {
                'url_regex': r'.*item.*',
                'xpaths': 
                {
                    'detail_price': r'//*[@id="detaliu-pret"]',
                    'detail_title': r'//h1[contains(@itemprop,"name")]',
                    'detail_description': r'//*[@id="j-product-description"]//div[contains(@class,"description-content")]',
                    'detail_image': r'//*[@id="magnifier"]/div/a/img'
                }
            },
            {
                'url_regex': r'.*category.*',
                'xpaths': 
                {
                    'list_container': r'//li[contains(@class,"list-item")]',
                    'list_title': r'//li[contains(@class,"list-item")]//a[contains(@class,"product")]',
                    'list_price': r'//li[contains(@class,"list-item")]//span[contains(@class,"price")]',
                    'list_image': r'//li[contains(@class,"list-item")]//img[contains(@itemprop,"image")]'
                }
            }
        ]

aliexpress_df = label_data(dfs['aliexpress'], rules)

In [19]:
aliexpress_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label          0
detail_title_label          0
detail_description_label    0
detail_image_label          0
list_container_label        0
list_title_label            0
list_price_label            0
list_image_label            0
dtype: int64

**NOTE:** Aliexpress employs heavy scrape-protection. Apparently no data was scraped, it might not be viable to use its data

#### Amazon

In [20]:
rules =  [
            {
                'url_regex': r'.*',
                'xpaths': 
                {
                    'detail_price': r'//*[@id="priceblock_ourprice"]',
                    'detail_title': r'//*[@id="productTitle"]',
                    'detail_description': r'//*[@id="productDescription"]',
                    'detail_image': r'//*[@id="landingImage"]'
                }
            },
            {
                'url_regex': r'.*',
                'xpaths': 
                {
                    'list_container': r'//div[contains(@class,"s-item-container")]',
                    'list_title': r'//div[contains(@class,"s-item-container")]//h2',
                    'list_price': r'//span[contains(@class,"-price-whole")]',
                    'list_image': r'//div[contains(@class,"s-item-container")]//img[contains(@class,"s-access-image")]'
                }
            }
        ]

amazon_df = label_data(dfs['amazon'], rules)

In [21]:
amazon_df.filter(axis='columns', regex=r'.*label').sum()

detail_price_label           31
detail_title_label            5
detail_description_label     27
detail_image_label            4
list_container_label        611
list_title_label            360
list_price_label            237
list_image_label            505
dtype: int64

### Wrapping up
Now, having labeled all the data, we ca export it to a CSV. The labels will be in the order of the `lxml` iteration of nodes on each page.

In [24]:
label_dfs = [emag_df, okazii_df, olx_df, aliexpress_df, amazon_df, lajumate_df, piataaz_df]
label_df = pd.concat(label_dfs, axis='rows', ignore_index=True)

In [25]:
label_df.to_csv('../data/ecommerce-labels.csv')